#Clasifica imágenes de los pisos en habitaciones


In [282]:
# Librerías y funciones para tensorflow 2.1.0.

!pip install tensorflow==2.1.0

import tensorflow as tf
import tensorflow_hub as hub

# Comprobamos la version del Tensorflow
print(tf.__version__)

# Para las descargas de imágenes.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# Para manipular la imagen.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# Manipulación de variables temporales.
import time

# Para comprobar GPU disponibles
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2.1.0
The following GPU devices are available: /device:GPU:0


In [283]:
import numpy as np 
import pandas as pd

# Conecta con google drive
from google.colab import drive
drive.mount('/content/drive')

# Carga del dataset de pisos de airbnb Madrid filtrado (13258 reg) sólo por la ciudad Madrid.
airbnbMadrid = pd.read_csv('/content/drive/My Drive/airbnblistingsMadridv1.csv', sep=';', decimal='.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Le pasamos la dirección url de la imagen y su tamaño
def download_and_resize_image(url, new_width=256, new_height=256):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  return filename

Elejimos un módulo de detección(FasterRCNN+InceptionResNet V2 o ssd+mobilenet V2) y lo usaremos en la imagen

In [285]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]
detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
# Pasamos la imagen a un tensorFlow
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

In [0]:
# Nos quedamos solo con la columna de las fotos del piso
airbnbMadridXlPicUrl = airbnbMadrid['XL Picture Url']

In [0]:
# Devuelve elementos en la foto
def room_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  elementos=[]
  colors = list(ImageColor.colormap.values())
  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()
# Buscamos uno a uno los elementos posibles en la foto
  for i in range(min(boxes.shape[0], max_boxes)):
    habitacion  = 'No existe clasificacion'
    if (scores[i] >= min_score) and (habitacion == 'No existe clasificacion'):
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      elementos.append(class_names[i].decode("ascii")) # Añadimos los elementos a lista
  return elementos # Devuelve la lista con los elementos de la foto

In [0]:
# Preparamos la imagen y ejecutamos la funcion que devolverá elementos de la hab
def run_detector2(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()
  result = {key:value.numpy() for key,value in result.items()}

  elementos_habitacion = room_boxes(img.numpy(), result["detection_boxes"],
            result["detection_class_entities"], result["detection_scores"])
  return elementos_habitacion

In [0]:
# Reduce el dataset a la variable objetivo : 
airbnbMadridXlPicUrl = airbnbMadrid['XL Picture Url']

In [291]:
# Se calcula el número de nulos
print(f'El dataframe tiene un total de {len(airbnbMadridXlPicUrl)} filas.')
print((f'Número de nulos : '),len(airbnbMadridXlPicUrl) - airbnbMadridXlPicUrl.count())

El dataframe tiene un total de 13258 filas.
Número de nulos :  2484


In [0]:
# Eliminacion de nulos y abreviatura del dataframe
airbnbMadridXlPicUrl = airbnbMadridXlPicUrl.fillna('NoDisponible')

In [293]:
airbnbMadridXlPicUrl.head(5)

0    https://a0.muscache.com/im/pictures/86864154/1...
1    https://a0.muscache.com/im/pictures/69897767/2...
2    https://a0.muscache.com/im/pictures/dbcf7393-b...
3    https://a0.muscache.com/im/pictures/59706828/a...
4    https://a0.muscache.com/im/pictures/20e6f47e-2...
Name: XL Picture Url, dtype: object

In [0]:
#Segun los elemenos que aparecen en la foto, asignamos la habitacion 
def asigna_habitacion(elemen_hab):
  habitacion = 'Habitacion no clasificada' # Valor por defecto
# Partimos que la única habitación con bañera o taza o bidet es el WC
  if 'Bathtub' in elemen_hab: 
      habitacion = 'WC'
  if 'Toilet' in elemen_hab: 
      habitacion = 'WC'
  if 'Bidet' in elemen_hab: 
      habitacion = 'WC'
# Partimos que la única habitación con Sofá o TV mesa de comedor es el salón
  if 'Sofa' in elemen_hab:
      habitacion = 'Salon'
  if 'Sofa Bed' in elemen_hab:
      habitacion = 'Salon'
  if 'Couch' in elemen_hab:
      habitacion = 'Salon'
  if 'Television' in elemen_hab:
      habitacion = 'Salon'
  if 'Studio couch' in elemen_hab:
      habitacion = 'Salon'
  if 'Kitchen & dining room table' in elemen_hab:
      habitacion = 'Salon'
# Partimos que la única habitación con cama es el dormitorio
  if 'Bed' in elemen_hab:
      habitacion = 'Dormitorio'
  print(habitacion)

# Partimos que la única habitación con Nevera o lavadora o microondas
  return habitacion

In [0]:
# Creamos dataframe para las fotos del piso y su tipo de habitacion
dfXlPic=pd.DataFrame(columns=['XLPictureUrl','HabitacionType'])
# Metemos las fotos 
dfXlPic['XLPictureUrl']=airbnbMadridXlPicUrl

In [0]:
# Reducimos el dataset a 100 registros, pues procesar 1000 registros lleva 2 horas(imagina 13271)
dfXlPic100=dfXlPic[:100]

In [297]:
#Rellenamos los campos por defecto
j=0
for j in range(len(dfXlPic100)):
  if dfXlPic100['XLPictureUrl'][j] != 'NoDisponible':
     dfXlPic100['HabitacionType'][j] = 'Imagen no disponible'
  else:  
     dfXlPic100['HabitacionType'][j] = 'Habitacion no clasificada'

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [298]:
# Pasamos todas las imagenes del dataset a la funcion, y devuelve la habitacion que es
i = 0

for i in range(len(dfXlPic100)):
  if dfXlPic100['XLPictureUrl'][i] != 'NoDisponible': #Siempre que tenga enlace a la foto
     start_time = time.time()
     image_path = download_and_resize_image(dfXlPic100['XLPictureUrl'][i], 640, 480)
     elementos_hab=run_detector2(detector, image_path)
     end_time = time.time()
     dfXlPic100['HabitacionType'][i]=asigna_habitacion(elementos_hab)
     print(dfXlPic100['XLPictureUrl'][i])
     print(i)
     i = i+1

Dormitorio
https://a0.muscache.com/im/pictures/86864154/1356abbf_original.jpg?aki_policy=x_large
0


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


Dormitorio
https://a0.muscache.com/im/pictures/69897767/28e73423_original.jpg?aki_policy=x_large
1
Dormitorio
https://a0.muscache.com/im/pictures/dbcf7393-bb26-40af-ab91-aaf50ef7b380.jpg?aki_policy=x_large
2
Dormitorio
https://a0.muscache.com/im/pictures/59706828/afecc7d6_original.jpg?aki_policy=x_large
3
Salon
https://a0.muscache.com/im/pictures/20e6f47e-2a1d-47ea-a814-d02e38efe8ca.jpg?aki_policy=x_large
4
Dormitorio
https://a0.muscache.com/im/pictures/e684a7f5-038f-48a9-9829-a347a9e76dd9.jpg?aki_policy=x_large
5
Salon
https://a0.muscache.com/im/pictures/39774047/4b1fac5d_original.jpg?aki_policy=x_large
6
Habitacion no clasificada
https://a0.muscache.com/im/pictures/49838903/35a630eb_original.jpg?aki_policy=x_large
7
Dormitorio
https://a0.muscache.com/im/pictures/e1eb98be-2800-4982-83f3-c566829ec973.jpg?aki_policy=x_large
8
Dormitorio
https://a0.muscache.com/im/pictures/13f63727-0893-416d-8efe-7d7ca4790739.jpg?aki_policy=x_large
9
Salon
https://a0.muscache.com/im/pictures/80655674/498

In [301]:
# El resultado es el siguiente :
dfXlPic100.head(10)

,XLPictureUrl,HabitacionType
0,https://a0.muscache.com/im/pictures/86864154/1...,Dormitorio
1,https://a0.muscache.com/im/pictures/69897767/2...,Dormitorio
2,https://a0.muscache.com/im/pictures/dbcf7393-b...,Dormitorio
3,https://a0.muscache.com/im/pictures/59706828/a...,Dormitorio
4,https://a0.muscache.com/im/pictures/20e6f47e-2...,Salon
5,https://a0.muscache.com/im/pictures/e684a7f5-0...,Dormitorio
6,https://a0.muscache.com/im/pictures/39774047/4...,Salon
7,https://a0.muscache.com/im/pictures/49838903/3...,Habitacion no clasificada
8,https://a0.muscache.com/im/pictures/e1eb98be-2...,Dormitorio
9,https://a0.muscache.com/im/pictures/13f63727-0...,Dormitorio


Hemos obtenido una clasificación que permitirá enviar un mensaje al dueño para sugerirle que suba fotos de las habitaciones que falten, para mejorar el servicio.